In [36]:
import requests
import re
import time
from lxml import html
from selenium import webdriver
from pyecharts import Map

r = requests.get(url='http://www.sohu.com/a/223885358_118392')    # 最基本的GET请求
# time.sleep(60)
html = r.text
content = re.findall(u'<p>([\u2E80-\u9FFF]{2,3})[u"市"u"省"u"特"u"壮"u"回"u"维"u"自"u"人"].{0,6}\uff08(\d*)\u540d\uff09 </p>.*?<p>(.*?)</p>', html, re.S)


In [37]:
import pandas as pd
# 取地区，姓名，性别，族
data = [(_[0],_[2]) for _ in content]
data[0]

('北京',
 '于志刚、马一德、王全、王铮、方复全、厉莉（女）、田春艳（女，满族）、冯乐平（女）、任鸣、伊彤（女，满族）、刘加军、刘伟、刘振所、齐玫（女，满族）、闫傲霜（女）、杜德印、李伟、李俊丰、李勇、李晓林、杨万明、杨元庆、吴素芳（女）、吴晨、邱勇、何福胜、张工、张礼斌、张建东、张硕辅、陈立人、陈吉宁、林建华、罗瀛（女，满族）、周立云、庞丽娟（女）、赵郁、赵晓燕（女）、侯湛莹（女）、秦飞、班宇侠（女，回族）、夏伟东、夏林茂、顾晋、徐滔（女）、高子程、谈绪祥、阎建国、敬大力、韩永进、程京、靳伟、雷军、蔡奇、戴天方 ')

In [38]:
def get_locality_name_sex_race(param):
    print('整理"'+ str(param[0]+'"地区代表数据'))
    print('--------------------------------')
    locality = param[0]    
    for el in param[1].split(u'、'):# '、'分割姓名
        temp = re.split(u'\uff08|\uff0c|\uff09', el)
        length = len(temp)
        if length == 1: # 只有名字，加上： 男 汉族
            temp += [u'男', u'汉族']
        elif length == 3: # 有名字 有性别，加上 汉族
            if len(temp[1]) == 1:
                temp[2] = u'汉族'
            else: # 有名字 有民族，加上 男
                temp[2] = temp[1]
                temp[1] = u'男'
        elif length == 4:# ？
            temp.pop()
            if u'族' not in temp[2]:
                temp[2] = u'汉族'                
        df.loc[df.shape[0]] = [locality]+temp


# 加上list（）就可以正常显示了，为什么呢？

In [39]:
df = pd.DataFrame(columns=('locality', 'name', 'sex', 'race'))#生成空的pandas表
list(map(get_locality_name_sex_race, data))
df.head()

整理"北京"地区代表数据
--------------------------------
整理"天津"地区代表数据
--------------------------------
整理"河北"地区代表数据
--------------------------------
整理"山西"地区代表数据
--------------------------------
整理"内蒙古"地区代表数据
--------------------------------
整理"辽宁"地区代表数据
--------------------------------
整理"吉林"地区代表数据
--------------------------------
整理"黑龙江"地区代表数据
--------------------------------
整理"上海"地区代表数据
--------------------------------
整理"江苏"地区代表数据
--------------------------------
整理"浙江"地区代表数据
--------------------------------
整理"安徽"地区代表数据
--------------------------------
整理"福建"地区代表数据
--------------------------------
整理"江西"地区代表数据
--------------------------------
整理"山东"地区代表数据
--------------------------------
整理"河南"地区代表数据
--------------------------------
整理"湖北"地区代表数据
--------------------------------
整理"湖南"地区代表数据
--------------------------------
整理"广东"地区代表数据
--------------------------------
整理"广西"地区代表数据
--------------------------------
整理"海南"地区代表数据
--------------------------------
整理"重庆"地区代表数据
-------------------

,locality,name,sex,race
0,北京,于志刚,男,汉族
1,北京,马一德,男,汉族
2,北京,王全,男,汉族
3,北京,王铮,男,汉族
4,北京,方复全,男,汉族


In [40]:
def get_attr_sex_v(race_groupby_sex):
    attr_sex = race_groupby_sex.count().index.values.tolist()
    attr_v = race_groupby_sex.count().values[0:2,0].tolist()
    return attr_sex,attr_v

In [41]:
from pyecharts import Bar
race_groupby_sex = df[df.race==u'汉族'].groupby(df['sex'])
attr_sex,attr_v = get_attr_sex_v(race_groupby_sex)
bar = Bar("", "", width=600, height=400)
bar.add("汉族", attr_sex, attr_v, is_more_utils=True)
race_groupby_sex = df[df.race!=u'汉族'].groupby(df['sex'])
minorities_sex,minorities_v = get_attr_sex_v(race_groupby_sex)
bar.add("少数民族", minorities_sex, minorities_v, is_more_utils=True)
# bar.render()
bar

In [42]:
from pyecharts import Pie
pie = Pie("")
pie.add("", ['汉族女','汉族男','少数民族女','少数民族男'], attr_v+minorities_v, is_label_show=True, is_legend_show=False)
# pie.render()
pie

In [43]:
from pyecharts import Bar

race_nums = df['race'].value_counts()
race_name = race_nums.index.values.tolist()
race_values = race_nums.values.tolist()
bar = Bar("", "少数民族")
bar.add("少数民族", race_name[1:], race_values[1:], 
        is_label_show=True,is_more_utils=True)

bar.render()
bar

In [44]:
from pyecharts import WordCloud # 写到最后，变量在上一个里面定义的

wordcloud = WordCloud(width=1300, height=620)
wordcloud.add("", race_name, race_values, word_size_range=[50, 100])# 原帖有误
wordcloud.render()
wordcloud